In [23]:
import os, sys, requests, itertools
import numpy as np
from bs4 import BeautifulSoup

module_path = os.path.abspath(os.path.join('./utils'))
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
    print(sys.path)
    
import imdbUtils as imdbutils


/home/jjkh/repos/recommender_system/utils


In [6]:
#!python -m pip install requests

In [7]:
min_rating, max_rating = 4.0, 10.0
n_votes = 50000
genres = 'thriller'
title_count = 250

SEARCH_URL = f"https://www.imdb.com/search/title/?title_type=feature&user_rating={min_rating},{max_rating}&num_votes={n_votes},&genres={genres}&view=simple&sort=user_rating,desc&count={title_count}"

In [9]:
movies_soup = imdbutils.getSoup(SEARCH_URL)

In [11]:
# find all a-tags with class:None
movie_tags = movies_soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]

There are a total of 250 movie titles
Displaying 10 titles


['/title/tt0468569/',
 '/title/tt1375666/',
 '/title/tt6751668/',
 '/title/tt0114369/',
 '/title/tt0102926/',
 '/title/tt0482571/',
 '/title/tt0407887/',
 '/title/tt0114814/',
 '/title/tt0110413/',
 '/title/tt0054215/']

In [14]:
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 10 user reviews links")
movie_links[:10]

There are a total of 250 movie user reviews
Displaying 10 user reviews links


['https://www.imdb.com/title/tt0468569/reviews',
 'https://www.imdb.com/title/tt1375666/reviews',
 'https://www.imdb.com/title/tt6751668/reviews',
 'https://www.imdb.com/title/tt0114369/reviews',
 'https://www.imdb.com/title/tt0102926/reviews',
 'https://www.imdb.com/title/tt0482571/reviews',
 'https://www.imdb.com/title/tt0407887/reviews',
 'https://www.imdb.com/title/tt0114814/reviews',
 'https://www.imdb.com/title/tt0110413/reviews',
 'https://www.imdb.com/title/tt0054215/reviews']

In [15]:
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 10 user reviews links")
movie_links[:10]

There are a total of 250 movie user reviews
Displaying 10 user reviews links


['https://www.imdb.com/title/tt0468569/reviews',
 'https://www.imdb.com/title/tt1375666/reviews',
 'https://www.imdb.com/title/tt6751668/reviews',
 'https://www.imdb.com/title/tt0114369/reviews',
 'https://www.imdb.com/title/tt0102926/reviews',
 'https://www.imdb.com/title/tt0482571/reviews',
 'https://www.imdb.com/title/tt0407887/reviews',
 'https://www.imdb.com/title/tt0114814/reviews',
 'https://www.imdb.com/title/tt0110413/reviews',
 'https://www.imdb.com/title/tt0054215/reviews']

In [21]:
# get a list of soup objects
movie_soups = [imdbutils.getSoup(link) for link in movie_links]

# get all 500 movie review links
movie_review_list = [imdbutils.getReviews(movie_soup) for movie_soup in movie_soups]

movie_review_list = list(itertools.chain(*movie_review_list))
print(len(movie_review_list))

print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
print("Displaying 10 reviews")
movie_review_list[:10]

500
There are a total of 500 individual movie reviews
Displaying 10 reviews


['https://www.imdb.com/review/rw6513945/',
 'https://www.imdb.com/review/rw5478826/',
 'https://www.imdb.com/review/rw2377775/',
 'https://www.imdb.com/review/rw4692192/',
 'https://www.imdb.com/review/rw5281302/',
 'https://www.imdb.com/review/rw5195256/',
 'https://www.imdb.com/review/rw0960802/',
 'https://www.imdb.com/review/rw0370669/',
 'https://www.imdb.com/review/rw5797530/',
 'https://www.imdb.com/review/rw1198894/']

In [25]:
# get review text from the review link
review_texts = [imdbutils.getReviewText(url) for url in movie_review_list]

# get movie name from the review link
movie_titles = [imdbutils.getMovieTitle(url) for url in movie_review_list]

# label each review with negative or positive
review_sentiment = np.array(['negative', 'positive'] * (len(movie_review_list)//2))

# construct a dataframe
df = pd.DataFrame({'movie': movie_titles, 'user_review_permalink': movie_review_list,
             'user_review': review_texts, 'sentiment': review_sentiment})

NameError: name 'pd' is not defined

In [ ]:
MOVIES_DIR = 
REVIEWS_DIR =
'https://shravan-kuchkula.github.io/scrape_imdb_movie_reviews/#step-3-for-each-movie-extract-the-movies-user-reviews-link'